# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy
import geoviews
import pyproj


# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,14.20,83,100,2.06,US,1697681134
1,1,adamstown,-25.0660,-130.1015,22.67,75,17,0.50,PN,1697681134
2,2,port elizabeth,-33.9180,25.5701,15.73,95,20,2.57,ZA,1697681001
3,3,waitangi,-43.9535,-176.5597,13.23,81,99,3.13,NZ,1697681134
4,4,alofi,-19.0595,-169.9187,25.94,53,40,8.75,NU,1697681134


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display
# Configure the map plot

all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = 'Country')
# Display the map
all_city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp0 = city_data_df.loc[city_data_df['Max Temp']<27,:]
city_temp = city_temp0.loc[city_temp0['Max Temp']>21,:]
city_wind = city_temp.loc[city_temp['Wind Speed']<4.5,:]
city_cloud = city_wind.loc[city_wind['Cloudiness']==0,:]
city_cloud

# Drop any rows with null values
city_ideal = city_cloud.dropna()

# Display sample data
city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,remire-montjoly,4.9167,-52.2667,24.02,100,0,0.00,GF,1697681135
45,45,naze,28.3667,129.4833,23.51,51,0,1.27,JP,1697681139
64,64,tura,25.5198,90.2201,23.37,77,0,0.52,IN,1697681141
90,90,lompoc,34.6391,-120.4579,24.08,94,0,2.06,US,1697680880
211,211,san patricio,28.0170,-97.5169,24.91,65,0,4.03,US,1697681158
289,289,al mutayrifi,25.4788,49.5576,24.19,68,0,3.56,SA,1697681168
296,296,tando adam,25.7637,68.6610,26.14,44,0,3.09,PK,1697681169
318,318,ivins,37.1686,-113.6794,26.62,26,0,2.24,US,1697681171
356,356,waddan,29.1614,16.1390,22.75,24,0,3.33,LY,1697681176
363,363,agadez,19.7500,10.2500,24.77,18,0,2.85,NE,1697681177


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
9,remire-montjoly,GF,4.9167,-52.2667,100,
45,naze,JP,28.3667,129.4833,51,
64,tura,IN,25.5198,90.2201,77,
90,lompoc,US,34.6391,-120.4579,94,
211,san patricio,US,28.0170,-97.5169,65,
289,al mutayrifi,SA,25.4788,49.5576,68,
296,tando adam,PK,25.7637,68.6610,44,
318,ivins,US,37.1686,-113.6794,26,
356,waddan,LY,29.1614,16.1390,24,
363,agadez,NE,19.7500,10.2500,18,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
remire-montjoly - nearest hotel: No hotel found
naze - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
al mutayrifi - nearest hotel: No hotel found
tando adam - nearest hotel: No hotel found
ivins - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
agadez - nearest hotel: No hotel found
garachico - nearest hotel: No hotel found
bihpuriagaon - nearest hotel: No hotel found
badarpur - nearest hotel: No hotel found
daudnagar - nearest hotel: No hotel found
gafargaon - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
bairnsdale - nearest hotel: No hotel found
bela - nearest hotel: No hotel found
taranagar - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found
hamirpur - nearest hotel: No hotel found
jalpaiguri - nearest hotel: No hotel found
bali chak - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
9,remire-montjoly,GF,4.9167,-52.2667,100,No hotel found
45,naze,JP,28.3667,129.4833,51,No hotel found
64,tura,IN,25.5198,90.2201,77,No hotel found
90,lompoc,US,34.6391,-120.4579,94,No hotel found
211,san patricio,US,28.0170,-97.5169,65,No hotel found
289,al mutayrifi,SA,25.4788,49.5576,68,No hotel found
296,tando adam,PK,25.7637,68.6610,44,No hotel found
318,ivins,US,37.1686,-113.6794,26,No hotel found
356,waddan,LY,29.1614,16.1390,24,No hotel found
363,agadez,NE,19.7500,10.2500,18,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)